## Intro

This notebook contains basic importing and light formatting of data to support my F1 Predictor project. The data below is queried from the [Egrast API](http://ergast.com/mrd/).

The Ergast database contains information on races, drivers, qualifying and final grand prox results, and more going back to the 1950s. Some features have more historical data than others for a variety of reasons. For example, though the first F1 World Championship was in 1950, the first Constructor's Champion wasn't crowned until 1958.

Importantly, Ergast rate-limits the API calls you can make. You must specify the number of rows to be returned with a limit of 1000. You can also offset the starting point of those rows via query string. In my project, I'm interested in making predictions about current F1 drivers and teams - so I attach query strings to fetch the most recent race data, going back to 2008. 

### Races

The first table to import is Ergast's base list of races (aka, Grand Prix events) per season. This is the top-most level of Ergast's database, "Race Schedule".

In [53]:
# import the basic packages, including Fast F1
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings("ignore")

In [49]:
df_races = pd.read_csv('../csv/races.csv')
df_races = df_races.drop('url', 1)
df_races

,raceId,year,round,circuitId,name,date,time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00
...,...,...,...,...,...,...,...
1075,1092,2022,19,22,Japanese Grand Prix,2022-10-09,05:10:00
1076,1093,2022,20,69,United States Grand Prix,2022-10-23,19:00:00
1077,1094,2022,21,32,Mexico City Grand Prix,2022-10-30,19:00:00
1078,1095,2022,22,18,São Paulo Grand Prix,2022-11-13,17:00:00


In [50]:
# Add useful info for each race, such as circuitName (which in F1 terms is often but not always synonomous with 
# the raceName. For example, the United States Grand Prix has been held at six different circuits over the years.)

df_circuits = pd.read_csv('../csv/circuits.csv')
df_races.merge(df_circuits[['circuitRef', 'country', 'circuitId']], on = 'circuitId', how = 'left')


,raceId,year,round,circuitId,name,date,time,circuitRef,country
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,albert_park,Australia
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,sepang,Malaysia
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,shanghai,China
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,bahrain,Bahrain
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,catalunya,Spain
...,...,...,...,...,...,...,...,...,...
1075,1092,2022,19,22,Japanese Grand Prix,2022-10-09,05:10:00,suzuka,Japan
1076,1093,2022,20,69,United States Grand Prix,2022-10-23,19:00:00,americas,USA
1077,1094,2022,21,32,Mexico City Grand Prix,2022-10-30,19:00:00,rodriguez,Mexico
1078,1095,2022,22,18,São Paulo Grand Prix,2022-11-13,17:00:00,interlagos,Brazil


### Results

The results of a race include the grid position of each driver (a grid position of 1 indicates the winner) as well as important race data including each driver's fastest lap time, average lap time, and whether or not they successfully completed the course. I add in some helpful identifying info from other parts of the database, such as driver name, constructor, and what the status codes mean. Status indictaes whether or not a driver completed the race or, if not, what the cause of the failure was.

In [51]:
df_constructors = pd.read_csv('../csv/constructors.csv')
df_drivers = pd.read_csv('../csv/drivers.csv')
df_status = pd.read_csv('../csv/status.csv')
df_results = pd.read_csv('../csv/results.csv')
df_results.merge(df_drivers[['driverRef', 'dob', 'driverId']], on = 'driverId', how = 'left')
df_results.merge(df_constructors[['constructorRef', 'constructorId']], on = 'constructorId', how = 'left')
df_results.merge(df_status[['status', 'statusId']], on = 'statusId', how = 'left')

# cleanup - some fields that are pure identifiers and no longer needed (i.e. 
# we can safely assume livery number is not relevant)
df_results.drop(['positionText', 'number'], 1)

,resultId,raceId,driverId,constructorId,grid,position,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,5,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,11,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,3,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25395,25401,1073,849,3,16,\N,16,0.0,50,\N,\N,30,15,1:29.293,212.912,3
25396,25402,1073,841,51,14,\N,17,0.0,33,\N,\N,33,16,1:29.442,212.557,6
25397,25403,1073,847,3,17,\N,18,0.0,26,\N,\N,23,19,1:30.647,209.732,6
25398,25404,1073,8,51,18,\N,19,0.0,25,\N,\N,23,18,1:29.698,211.951,23


### Qualifying

Qualifying refers to the performance of drivers in the preliminary competitions of a race, which typically determines the starting position of each driver for the final grand prix race on the final day of the round.

In [52]:
df_qual = pd.read_csv('../csv/qualifying.csv')
df_qual

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
9130,9171,1073,849,3,6,16,1:24.338,\N,\N
9131,9172,1073,847,3,63,17,1:24.423,\N,\N
9132,9173,1073,8,51,7,18,1:24.779,\N,\N
9133,9174,1073,854,210,47,19,1:24.906,\N,\N
